# [Std 04] - calculate Christoffel symbols in spherical coords

In [1]:
%display latex
import numpy as np
import scipy
from numpy import r_

First, let's define a Euclidean space in spherical coordinates, over the variables $r$, $\theta$, and $\phi$.  

In [19]:
E.<r, θ, φ> = EuclideanSpace(coordinates='spherical')
E.spherical_coordinates().coord_range()

r: (0, +oo); θ: (0, pi); φ: [0, 2*pi] (periodic)

To be complete, let's also check out its frames.  These are in order of its coordinate frame (a tuple of its basis and partials), it's basis frame (in spherical coordinates), which is derived as a Chart from the Cartesian coordinates.  Coordinate frames matter in GR, we will address that later. 

In [27]:
E.frames()

[Coordinate frame (E^3, (∂/∂r,∂/∂θ,∂/∂φ)),
 Vector frame (E^3, (e_r,e_θ,e_φ)),
 Coordinate frame (E^3, (e_x,e_y,e_z))]

* c.ref. https://math.stackexchange.com/q/1985964 and https://mathworld.wolfram.com/SphericalCoordinates.html; order of the parameter definitions matter!

* Given a curvilinear world-line path in spherical coordinates $ds^2 = dr^2 + r\ d \theta^2 + r^2 \sin^2 \theta\ d\phi^2$, we define a metric
    $$ g^{ij} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & r^2 & 0 \\ 0 & 0 & r^2 \sin^2\theta \end{bmatrix}$$

* We also define the Christoffel symbols in summation-notation: $$ \Gamma_{jk}^{i} \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \frac{\partial g^{kl}}{\partial x^j} + \frac{\partial g^{jl}}{\partial x^k} - \frac{\partial g^{jk}}{\partial x^l} \right] $$ or equivalently $$ \Gamma_{jk}^i \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \partial_j g^{kl} + \partial_k g^{jl} - \partial_l g^{jk} \right] $$

* **QUESTION :>** in the definitions above, are the coordinate indices supposed to be upper or lower? Is the definition valid for the contravariant, or the covariant, expression of the indices? Differing definitions across sources. 

In [36]:
from sage.tensor.modules.tensor_with_indices import TensorWithIndices

i,j,k,l = var('i,j,k,l')

# For our variable names, we'll use Unicode U+2054 (CHARACTER TIE) to 
#  indicate contravariant (upper) indices, followed by an underscore preceding
#  covariant indices.
# For convenience, I've copied over the unicode characters here, e.g.: g⸢ij⸣⸤kl⸥ 
g⁀ij = Manifold(3, "g^{ij}", )

* recreate the same thing, but use the manifold definition so I can use the tensor transform to calculate the christoffel symbols from that

* use the spherical coordinates of the manifold defined for the rank (0,4) tensor to calculate non-zero christoffel symbols using numpy einsum. 